In [1]:
# If not installed, run this cell. It is needed for access to GEE
#!pip install ee
#!pip show ee

In [12]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-jvg')

In [3]:
# If not installed, run this cell. It helps visualization of data
#!pip install geemap
#!pip show geemap

In [14]:
import geemap
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [15]:
# Access an image asset from my account to test the code
#guate_bdry = ee.FeatureCollection('users/jvg/guatemala_buffer')
#guate_geom = guate_bdry.geometry()
#Map.centerObject(guate_geom)
#Map.addLayer(guate_bdry, {"color": 'red'}, 'Guatemala Boundary')

In [16]:
# Test continuation
#clay = ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02')
#clay_clipped = clay.clip(guate_bdry)
#Map.addLayer(clay_clipped, {}, 'clay_clipped')

In [17]:
# Initial parameters
warmup_yrs = 2
start_date = ''
amazon_basin_id = 6030007000

In [18]:
# HydroSHEDS v1 will be used. Specifically, its HydroBASINS
hydrobasins = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_3')
zoi = hydrobasins.select("HYBAS_ID").filter(ee.Filter.eq("HYBAS_ID", amazon_basin_id))
visualization = {
  "color": '808080',
  "strokeWidth": 1
}
Map.centerObject(zoi.geometry())
Map.addLayer(zoi, visualization, 'Amazon Basin')

In [19]:
# TerraClimate layers
terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filter(ee.Filter.date('2017-07-01', '2017-10-01'))
prec = terraclimate.select('pr')
pet = terraclimate.select('pet') # Scale factor: 0.1
runoff = terraclimate.select('ro')

In [21]:
whc = ee.Image('projects/ee-jvg/assets/whc_fc33')
whc = whc.clip(zoi)
Map.addLayer(whc, {}, 'Water Holding Capacity')

In [ ]:
#############################
# HELPFUL CODES
#############################

# To know how many bands an ImageCollection has
#terraclimate.size().getInfo()

# To get the attrIbutes (columns) as pandas dataframe
#geemap.ee_to_df(hydrobasins)

In [ ]:
# Let's import other libraries
import pandas as pd